In [3]:
import pandas as pd
import numpy as np
import json
import collections
import re
import category_encoders as ce

In [4]:
# Transform into ratio
cols_pre = [
'connection_period_initial',
'renewvia_id', 'country',
'nigeria_community', 'kenya_community', 'age', 'gender',
'occupation', 'primary_provider', 'primary_provider_occupation',
'employement_type', 'avg_household_income', 'household_headcount',
'adult_headcount', 'girls_headcount', 'boys_headcount', 'girls_age',
'boys_age', 'girls_schooling', 'girls_unschooled_reasons',
'boys_schooling',
'boys_unschooled_reasons', 
'household_business_owners', 'minigrid_signup_primary_reason',
'minigrid_signup_secondary_reason', 'power_sources',
'power_sources_usage',
'power_sources_primary',
'appliances_count', 'cellphones_count', 'appliances_type',
'appliances_explain', 'appliances_addition_type',
'light_hours_current',
'light_primary_sources', 'kerosene_lamps_count',
'kerosene_lamp_usage_time', 'kerosene_lamps_cost',
'cooking_energy_sources', 
'cooking_fuel_collection_time', 'cooking_fuel_responsible',
'cooking_energy_cost', 'applicances_charging_sources',
'applicances_charging_cost',
'feel_safe_dark', 'community_lights', 'home_exterior_lights',
'feel_safe_if_exterior_lights', 'feel_unsafe_reasons',
'phone_charge_location',
'phone_charge_frequency',
'phone_charge_cost', 'phone_charge_travel_distance', 'water_source',
'clean_drinking_water', 'clean_drinking_water_source',
'clean_water_source', 'water_collection_travel_distance',
'water_collection_time', 'water_collection_responsible',
'avg_person_age_water_collection', 'water_cost',
'other_household_activities', 'clinic_travel_distance',
'clinic_electricity_access',
'clinic_refrigeration_access', 'end_date', 'customerAccountNumber',
'tariff', 
'occupation_secondary_provider',
'cooking_energy_sources', 'community_clean_water_source'
]

#Importing the datasets
df = pd.read_csv("datasets_annotated/initial_annotated.csv", usecols=cols_pre)
# standardize case
df = df.apply(lambda x: x.astype(str).str.lower())

# Count of multiple selection columns

df['household_business_owners'] = df['household_business_owners'].map(lambda x: x.replace("none;", ""))
df['business_owners_count'] = df['household_business_owners'].map(lambda x: 
                                                                  len(x[:-1].split(";")) 
                                                                  if x != ""
                                                                  else 0)
df['business_owners_female'] = df['household_business_owners'].map(lambda x: 1 
                                                                            if x != ""
                                                                            and ('female' in x) 
                                                                            else 0)
df['appliances_type'] = df['appliances_type'].map(lambda x: x.replace(" ", ";"))
df['appliances_type_count'] = df['appliances_type'].map(lambda x: 
                                                          len(x[:-1].split(";")) 
                                                          if x != ""
                                                          else 0)
df['connection_period_initial'].replace('1-3 months', 
                                        '0-3 months', 
                                        inplace=True)

df['avg_person_age_water_collection'].replace('15 years or older', 
                                              '15 -18 years old', 
                                              inplace=True)

df['clinic_travel_distance'].replace('less than 1km', 
                                      'less than 1 km', 
                                      inplace=True)

trim_cols = ['clean_drinking_water_source', 'phone_charge_travel_distance']
for c in trim_cols:
    df[c] = df[c].str.rstrip()
    
cols_num = ['avg_household_income', 'household_headcount', 
            'girls_schooling', 'boys_schooling', 'appliances_count', 
            'appliances_type_count',
            'cellphones_count', 'light_hours_current', 'kerosene_lamps_count',
            'business_owners_count', 'business_owners_female']

df[cols_num].head()

,avg_household_income,household_headcount,girls_schooling,boys_schooling,appliances_count,appliances_type_count,cellphones_count,light_hours_current,kerosene_lamps_count,business_owners_count,business_owners_female
0,nan,nan,nan,nan,nan,1,nan,nan,nan,1,0
1,27000.0,3.0,nan,1.0,5.0,2,3.0,5.0,nan,1,1
2,30000.0,9.0,2.0,4.0,5.0,4,8.0,5.0,nan,1,1
3,30000.0,2.0,nan,nan,4.0,4,2.0,5.0,nan,1,1
4,38000.0,7.0,2.0,3.0,5.0,5,7.0,6.0,nan,1,0


In [5]:
df.replace('nan', np.nan, inplace=True)
def replace_str(value):
    replace_chars = {'': [")", '.)', ',', '/', 'ikom'
                         'u\n', "'", ".0", ' years',],
                    '_': ['- ',' (', ' ', '-', "__"]
                    }

    for new, old in replace_chars.items():
        for c in old:
            if type(value) == str:
                value = value.replace(c, new).rstrip()
            else:
                continue

    return value

for col in list(df.columns):
    df[col] = df[col].apply(lambda x: replace_str(x))
    
    
# Convert to numeric columns
for col in list(df.columns):
    df[col] = df[col].apply(lambda x: replace_str(x))
    if col in cols_num:
        # if col == 'light_hours_current':
        #     df[col].replace({"8-12": "8", "3-4":"3"}, inplace=True)
        df[col] = df[col].astype(str)
        df[col] = df[col].str.extract('(\d+)', expand=False)
        df[col] = df[col].astype(float).astype('Int64')
    else:
        continue
    
# df[cols_num].head()
df.to_csv("datasets_clean/initial_clean.csv")

In [6]:
# news = []
# for col in list(df.columns):
#     uq = df[col].unique()
#     if len(uq) < 10:
#         print(col, uq)
#         # news.append(col)

# # # # news

In [8]:
# Encoding values
df_enc = df.copy()
cols = list(df.columns)
# mapping missing or mistyped answers
with open('col_answers_mapping.json', 'r') as f:
    col_answers_map = json.load(f)
    
answers = col_answers_map['col_answers_map']
n_col = len(answers)
    
c_map = []
for i in range(0, n_col):
    col = answers[i]['new_name']
    mapping = answers[i]['mapping']
    if col in cols and mapping != {} and col != 'household_business_owners':
        vals = list(mapping.values())
        new_vals = [replace_str(v.lower()) for v in vals]
        if "yes" in new_vals and "no" in new_vals:
            # c_map.append({col: })
            mapping = {'no':0, 'yes':1, 
                       'option 1': 0, 'option 2': 1}
            df_enc[col] = df_enc[col].replace(mapping)
            
        else:
            mapping = dict()
            rng = len(new_vals)
            for idx, val in enumerate(new_vals):
                mapping[val] = idx+1
                
            df_enc[col] = df_enc[col].replace(mapping)

df_enc.to_csv('datasets_encoded/initial_encoded.csv')